In [ ]:
import transformers
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

In [ ]:
data= pd.read_csv('train_m2.csv')
data

,label,review
0,1,let me go to bed nazi faggot maybe i can catch...
1,1,making fun of retarded kids is bad but using t...
2,0,i bet this negroid is in ghetto hell wishing h...
3,1,btw it awesome that we can repost comments aga...
4,0,curanis are nigger jews
...,...,...
931,1,if ur still on twitter tell carlton i said his...
932,1,when i first got on here and said i hate trump...
933,1,was macht der moslem wenn der zion gegen seine...
934,0,it is awful look at world demographics asians ...


In [ ]:
#refernce code from hugging face
model_name = 'Helsinki-NLP/opus-mt-en-ROMANCE'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

english_model = 'Helsinki-NLP/opus-mt-ROMANCE-en'
english_tokenizer = MarianTokenizer.from_pretrained(english_model)
english_model = MarianMTModel.from_pretrained(english_model)

def translate(texts, model, tokenizer, language="fr"):
    # Prepare the text data into appropriate format for the model
    template = lambda text: f"{text}" if language == "en" else f">>{language}<< {text}"
    src_texts = [template(text) for text in texts]

    # Tokenize the texts
    encoded = tokenizer.prepare_seq2seq_batch(src_texts, return_tensors='pt')
    
    # Generate translation using model
    translated = model.generate(**encoded)

    # Convert the generated tokens indices back into text
    translated_texts = tokenizer.batch_decode(translated, skip_special_tokens=True)
    
    return translated_texts

def back_translate(texts, source_lang="en", target_lang="fr"):
    
    # Translate from source to target language
    fr_texts = translate(texts, model, tokenizer, language=target_lang)

    # Translate from target language back to source language
    back_translated_texts = translate(fr_texts, english_model, english_tokenizer, language=source_lang)
    
    return back_translated_texts

In [ ]:
def back(text):
    en_texts = [text]
    aug1 = back_translate(en_texts, source_lang="en", target_lang="es")
    aug2 = back_translate(en_texts, source_lang="en", target_lang="fr")
    aug=[aug1[0],aug2[0]]
    aug=list(dict.fromkeys(aug))
    return aug


In [ ]:
aug_qa=[]
new_label=[]
ori_qa=[]
for i in  range(935): 
    aug_text = back(data['review'][i])
    for j in aug_text:
        if j != data['review'][i]:
            aug_qa.append(j)
            new_label.append(data['label'][i])
            ori_qa.append(data['review'][i])

In [ ]:
print(len(ori_qa))
print(len(aug_qa))
print(len(new_label))

In [ ]:
my_dict = { 'label':new_label, 'review':aug_qa,}
df = pd.DataFrame(my_dict)
df


In [ ]:
result = pd.concat([data,df],ignore_index=True)
result.to_csv('augment_test_m2.csv',index=False)